In [ ]:
import grpc
import swoq_pb2
import swoq_pb2_grpc
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
player_id = '6616b1c5bd0a697480a68319'

In [ ]:
with grpc.insecure_channel('localhost:5009') as channel:
    stub = swoq_pb2_grpc.TrainingStub(channel)
    startResponse = stub.StartGame(swoq_pb2.StartRequest(playerId=player_id))

print(f'{startResponse.result=}')

game_id = startResponse.gameId
print(f'{game_id=}')
height = startResponse.height
width = startResponse.width
print(f'{height=}, {width=}')

map = np.array(startResponse.state.map).reshape(height, width)
plt.figure()
plt.imshow(map)

In [ ]:
player_pos = np.argwhere(map == swoq_pb2.PLAYER)[0]
target_pos = np.array([height-2, width-1])
print(f'{player_pos=}, {target_pos=}')

diff = target_pos - player_pos
moves = np.ones((2,2))
direction = (np.sign(diff) == 1) * 1
print(f'{direction=}')
moves[0,direction[0]] = diff[0]
moves[1,direction[1]] = diff[1]
moves = moves.flatten()
moves_e = np.exp(moves)
moves_p = moves_e / np.sum(moves_e)
print(f'{moves_p=}')

directions = [swoq_pb2.NORTH, swoq_pb2.SOUTH, swoq_pb2.WEST, swoq_pb2.EAST]
action = np.random.choice(directions, p=moves_p)
print(f'{action=}')

with grpc.insecure_channel('localhost:5009') as channel:
    stub = swoq_pb2_grpc.TrainingStub(channel)
    moveResponse = stub.Move(swoq_pb2.MoveRequest(gameId=game_id, direction=action))

print(f'{moveResponse.result=}')
print(f'{moveResponse.state.finished=}')

map = np.array(moveResponse.state.map).reshape(height, width)
plt.figure()
plt.imshow(map)